In [1]:
import skweak.utils
from spacy import displacy
import os
import myutils as utils

In [13]:
data_path = 'results/train_corpus_IO.spacy'
docs = skweak.utils.docbin_reader(data_path, spacy_model_name='en_core_web_lg')
data = utils.load_json('data/train-soft.json')

In [14]:
docs = list(docs)

In [15]:
len(docs)

2407

In [26]:
sample_size = 50
i = 0
for doc, d in zip(docs, data):
    if i < sample_size:
        skweak.utils.display_entities(doc, 'hmm')
        print(d['label'])
    i+=1

PARTIAL_CORRECT


CORRECT


CORRECT


CORRECT


CORRECT


...

CORRECT


INCORRECT


INCORRECT


PARTIAL_CORRECT


CORRECT


In [21]:
relations_correct, relations_partial, relations_incorrect = [], [], []
len_rubrics_correct, len_rubrics_partial, len_rubrics_incorrect = [], [], []
len_rubrics_average_correct, len_rubrics_average_partial, len_rubrics_average_incorrect = [], [], []
for doc, d in zip(docs, data):
    c = an['label']
    if c == 'CORRECT':
        relations_correct.append(rel)
        len_rubrics_correct.append(len(len_rubrics))
        len_rubrics_average_correct.append(len_rubrics_average)
    elif c == 'PARTIAL_CORRECT':
        relations_partial.append(rel)
        len_rubrics_partial.append(len(len_rubrics))
        len_rubrics_average_partial.append(len_rubrics_average)
    elif c == 'INCORRECT':
        relations_incorrect.append(rel)
        len_rubrics_incorrect.append(len(len_rubrics))
        len_rubrics_average_incorrect.append(len_rubrics_average)

    
    spans = skweak.utils.get_spans(doc, ["hmm"])
    sum_annotated_spans +=len(spans)
    sum_tokens += len(doc)
    for span in spans:
        sum_annotated_tokens+= len(span)

In [25]:
print('annoteated spans:', sum_annotated_spans/len(docs))
print('annotated tokens:',sum_annotated_tokens/len(docs))
print('tokens per answer:', sum_tokens/len(docs))

annoteated spans: 1.870793518903199
annotated tokens: 28.638969671790612
tokens per answer: 48.345658496053176
